In [ ]:
prefix = 'lstm'

import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

In [ ]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

In [ ]:
WORK_DIRECTORY = 'training-data'
data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

In [ ]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = f'{account}.dkr.ecr.{region}.amazonaws.com/{prefix}:latest'

estimator = sage.estimator.Estimator(
    image, 
    role, 
    1, 
    'ml.c4.2xlarge', 
    output_path="s3://{}/output".format(sess.default_bucket()),
    sagemaker_session=sess)

estimator.fit(data_location)

In [ ]:
predictor = estimator.deploy(1, 'ml.m4.xlarge')

In [ ]:
l = list(range(1,101))
s = ",".join(map(str, l))
b = bytearray(s, "utf-8")

b_response = predictor.predict(b)

s_response = b_response.decode("utf-8") 
l_response = s_response.split()

In [ ]:
type(l_response)

In [ ]:
l_response